# Imports

In [1]:
import numpy as np
import os
import sys
import pickle
import scipy
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import matplotlib as plt
import statsmodels.api
import statsmodels as sm

## Helper Functions

In [2]:
%store -r analysis_hf
sys.path.insert(0, analysis_hf)

import nodeMapping as nm
import normalize as norm

# Set Directory Paths

In [3]:
# Directory path to load formatted Thickness Data
%store -r thick_dataDir
# Directory path to save Calcualted Analysis Data
%store -r thick_CalData
# Directory path to save the Generated Figures
%store -r thickness_Fig

# Loading

## Loading Volume W Score (TAU, TDP) - 400 Regions

In [4]:
# TAU_vol_w
with open(os.path.join(thick_CalData, 'TAU_vol_w.pkl'), 'rb') as f:
    TAU_vol_w = pickle.load(f)
f.close()

# TDP_vol_w
with open(os.path.join(thick_CalData, 'TDP_vol_w.pkl'), 'rb') as f:
    TDP_vol_w = pickle.load(f)
f.close()

# T-test between TAU vs TDP

In [5]:
# Number of regions of Pathology (N = 40)
N = TAU_vol_w.shape[1]

In [6]:
N

400

In [7]:
TAU_vol_w.shape

(26, 400)

In [8]:
TDP_vol_w.shape

(30, 400)

## T-test. Since the sample numbers are different between TAU and TDP, we would perform Welch's t-test

In [9]:
TAU_gt_TDP_Ttest_stat_list = []
TAU_gt_TDP_Ttest_pval_list = []

TDP_gt_TAU_Ttest_stat_list = []
TDP_gt_TAU_Ttest_pval_list = []


for i in range(N):
    TAU_data = TAU_vol_w[:, i]
    TDP_data = TDP_vol_w[:, i]

    # ignore NaN Values in T-test
    TAU_gt_TDP_Ttest = stats.ttest_ind(a=TAU_data, b=TDP_data, equal_var=False, 
                                       alternative='greater', nan_policy='omit')
    TDP_gt_TAU_Ttest = stats.ttest_ind(a=TAU_data, b=TDP_data, equal_var=False, 
                                       alternative='less', nan_policy='omit')

    TAU_gt_TDP_Ttest_stat = TAU_gt_TDP_Ttest[0]
    TAU_gt_TDP_Ttest_pval = TAU_gt_TDP_Ttest[1]

    TAU_gt_TDP_Ttest_stat_list.append(TAU_gt_TDP_Ttest_stat)
    TAU_gt_TDP_Ttest_pval_list.append(TAU_gt_TDP_Ttest_pval)

    TDP_gt_TAU_Ttest_stat = TDP_gt_TAU_Ttest[0]
    TDP_gt_TAU_Ttest_pval = TDP_gt_TAU_Ttest[1]

    TDP_gt_TAU_Ttest_stat_list.append(TDP_gt_TAU_Ttest_stat)
    TDP_gt_TAU_Ttest_pval_list.append(TDP_gt_TAU_Ttest_pval)
    

## Multiple Comparison Correction / alpha = 0.05 / Method: Benjamini/Hochberg (non-negative)

In [18]:
TAU_gt_TDP_Ttest_pval_list

[0.828599891759586,
 0.8819285195786994,
 0.15696118712473112,
 0.7401134050737037,
 0.09644113266160141,
 0.06585239111708767,
 0.9700285018172103,
 0.6840276434175299,
 0.044945294193286904,
 0.846584590176414,
 0.08524114369300226,
 0.32676723146502074,
 0.8340262652259405,
 0.5944392378103325,
 0.10406361291205185,
 0.6472824922299545,
 0.6462843183890173,
 0.14114970697140386,
 0.34752841113876065,
 0.46744079549361245,
 0.03641878972168078,
 0.3690815894795559,
 0.6152268384648992,
 0.24123584471128923,
 0.24790397552634214,
 0.03792978284967951,
 0.6626158882589928,
 0.5037587596010534,
 0.6149838579634486,
 0.5420645393876653,
 0.34845731201882224,
 0.7302159358614303,
 0.5130866065370845,
 0.030981143303184046,
 0.5553159259371797,
 0.10256417100696034,
 0.6821123984781214,
 0.5642412676148127,
 0.39773843657142,
 0.6581755386930204,
 0.05462875881652756,
 0.7957866466845535,
 0.3829502642228034,
 0.5705545076312049,
 0.1630204274348258,
 0.03827417385134887,
 0.47439536519298

### Corrected for TAU > TDP

In [19]:
TAU_gt_TDP_reject, TAU_gt_TDP_pvals_corrected, alphacSidak, alphacBonf = sm.stats.multitest.multipletests(TAU_gt_TDP_Ttest_pval_list, alpha=0.05,
                                                                                                           method='fdr_bh', is_sorted=False, 
                                                                                                           returnsorted=False)

In [20]:
TAU_gt_TDP_pvals_corrected

array([0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960693,
       0.99960693, 0.99960693, 0.99960693, 0.99960693, 0.99960

### Corrected for TDP > TAU

In [11]:
TDP_gt_TAU_reject, TDP_gt_TDP_pvals_corrected, alphacSidak, alphacBonf = sm.stats.multitest.multipletests(TDP_gt_TAU_Ttest_pval_list, alpha=0.05,
                                                                                                          method='fdr_bh', is_sorted=False, 
                                                                                                          returnsorted=False)

## TAU > TDP

In [12]:
# Pandas DataFrame
TAU_gt_TDP_df = pd.DataFrame(np.concatenate((np.array(TAU_gt_TDP_pvals_corrected).reshape((-1, 1)), 
                                             np.array(TAU_gt_TDP_reject).reshape((-1, 1))), axis=1), 
                                             columns=['corrected pval', 'Result'])

TAU_gt_TDP_df['Result'] = TAU_gt_TDP_df['Result'].astype('bool')

In [13]:
TAU_gt_TDP_df

,corrected pval,Result
0,0.999607,False
1,0.999607,False
2,0.999607,False
3,0.999607,False
4,0.999607,False
...,...,...
395,0.999607,False
396,0.999607,False
397,0.999607,False
398,0.999607,False


## TDP > TAU

In [14]:
# Pandas DataFrame
TDP_gt_TAU_df = pd.DataFrame(np.concatenate((np.array(TDP_gt_TDP_pvals_corrected).reshape((-1, 1)), 
                                             np.array(TDP_gt_TAU_reject).reshape((-1, 1))), axis=1), 
                                             columns=['corrected pval', 'Result'])

TDP_gt_TAU_df['Result'] = TDP_gt_TAU_df['Result'].astype('bool')

In [15]:
TDP_gt_TAU_df

,corrected pval,Result
0,0.382904,False
1,0.314857,False
2,0.923878,False
3,0.479054,False
4,0.953624,False
...,...,...
395,0.728061,False
396,0.371417,False
397,0.395798,False
398,0.594183,False


# Save T-test results as csv

In [16]:
# Original
TAU_gt_TDP_df.to_csv(thickness_Fig + '/TAU_gt_TDP_Ttest(Vol_WScore)_400.csv', index=True)

TDP_gt_TAU_df.to_csv(thickness_Fig + '/TDP_gt_TAU_Ttest(Vol_WScore)_400.csv', index=True)